<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2023-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

<h2 style="color:#03122E;text-align:center;"> Capitulo 2. Metodos para Ecuaciones Diferenciales Parciales Elipticas<br> </h2>
<h3 style="color:#03122E;text-align:center;">             Metodos de Elementos Finitos <br> </h3>
<h3 style="color:#03122E;text-align:center;">             Continuum Mechanics <br> </h3>
<hr style="border:3px solid #E3AE00 "> </hr>

In [4]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.geom2d import SplineGeometry


In [38]:
def stress(u):
    return lambda_par*(div(u))*Id(2) + 2*mu_par*strain(u)

def strain(u):
    return 0.5*(grad(u) + grad(u).trans)

In [34]:
geo = SplineGeometry()

pnts =[(0,0),(5,0),(5,1),(0,1)]
p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
          [["line",p2,p3],"right"],
          [["line",p3,p4],"top"],
          [["line",p4,p1],"left"]]
[geo.Append(c,bc=bc, leftdomain=1, rightdomain=0) for c,bc in curves]
geo.AddCircle(c=(2.5,0.5),r=0.25,maxh=0.05,bc="circle",leftdomain=0,rightdomain=1)
ngmesh = geo.GenerateMesh(maxh=0.1)
mesh = Mesh(ngmesh)
Draw(mesh)

bdry_values = {'left': CoefficientFunction((0,0)), 'right': CoefficientFunction((1,0))}
values_list = [bdry_values[bc]
               if bc in bdry_values.keys() else ((0,0))
               for bc in mesh.GetBoundaries()]
g = CoefficientFunction(values_list)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [35]:
# H1-conforming finite element space
fes = VectorH1(mesh, order=1, dirichlet='left|right')

# define trial- and test-functions
u = fes.TrialFunction()
v = fes.TestFunction()

# the right hand side
force = CoefficientFunction((0,0))

lf = LinearForm(fes)
lf += force * v * dx

# the bilinear-form 
a = BilinearForm(fes, symmetric=True)
a += lambda_par*div(u)*div(v)*dx + 2*mu_par*InnerProduct(strain(u),strain(v))*dx

a.Assemble()
lf.Assemble()

# the solution field 
gfu = GridFunction(fes)
gfu.Set(g, definedon=mesh.Boundaries('left|right'))
r = lf.vec.CreateVector()
r.data = lf.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r


In [33]:
Draw(gfu, deformation=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

# Perdida de la Coercividad

In [70]:
geo = SplineGeometry()

pnts =[(0,0),(3,0),(3,1),(0,1)]
p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
          [["line",p2,p3],"right"],
          [["line",p3,p4],"top"],
          [["line",p4,p1],"left"]]
[geo.Append(c,bc=bc, leftdomain=1, rightdomain=0) for c,bc in curves]
geo.AddCircle(c=(1.5,0.5),r=0.25,maxh=0.025,bc="circle2",leftdomain=0,rightdomain=1)
geo.AddCircle(c=(0.625,0.5),r=0.2,maxh=0.025,bc="circle1",leftdomain=0,rightdomain=1)
geo.AddCircle(c=(2.375,0.5),r=0.2,maxh=0.025,bc="circle3",leftdomain=0,rightdomain=1)

ngmesh = geo.GenerateMesh(maxh=0.1)
mesh = Mesh(ngmesh)
Draw(mesh)

bdry_values = {'left': CoefficientFunction((0,0)), 'right': CoefficientFunction((1,0))}
values_list = [bdry_values[bc]
               if bc in bdry_values.keys() else ((0,0))
               for bc in mesh.GetBoundaries()]
g = CoefficientFunction(values_list)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [71]:
lambda_par = Parameter(100.0) # Ratio lambda/mu = 100
mu_par = Parameter(1.0)
def stress(u):
    return lambda_par*(div(u))*Id(2) + 2*mu_par*strain(u)

def strain(u):
    return 0.5*(grad(u) + grad(u).trans)
# H1-conforming finite element space
fes = VectorH1(mesh, order=1, dirichlet='left|right')

# define trial- and test-functions
u = fes.TrialFunction()
v = fes.TestFunction()

# the right hand side
force = CoefficientFunction((0,0))

lf = LinearForm(fes)
lf += force * v * dx

# the bilinear-form 
a = BilinearForm(fes, symmetric=True)
a += lambda_par*div(u)*div(v)*dx + 2*mu_par*InnerProduct(strain(u),strain(v))*dx

a.Assemble()
lf.Assemble()

# the solution field 
gfu = GridFunction(fes)
gfu.Set(g, definedon=mesh.Boundaries('left|right'))
r = lf.vec.CreateVector()
r.data = lf.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r


In [72]:
Draw(gfu, deformation=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [73]:
sigma = grad(gfu)#.MakeVariable()

def sigma_min(sigma): # magnitude of first principal stress
    return((sigma[0,0]+sigma[1,1])/2 - sqrt(((sigma[0,0]-sigma[1,1])/2)**2 + sigma[1,0]**2))

def sigma_max(sigma): # magnitude of first principal stress
    return((sigma[0,0]+sigma[1,1])/2 + sqrt(((sigma[0,0]-sigma[1,1])/2)**2 + sigma[1,0]**2))

Draw(sigma_max(sigma), mesh)
Draw(sigma_min(sigma), mesh)
Tresca = sigma_max(sigma)-sigma_min(sigma)
Draw(Tresca, mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [74]:
help(Draw)

Help on function Draw in module ngsolve.webgui:

Draw(mesh_or_func, mesh_or_none=None, name='function', order=2, min=None, max=None, draw_vol=True, draw_surf=True, autoscale=True, intpoints=None, deformation=False, interpolate_multidim=False, animate=False, clipping=None, vectors=None, js_code=None, eval_function=None, eval=None, filename='', objects=[], nodal_p1=False, settings={})

